In [ ]:

import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2


class RouteFinder:
    def __init__(self, search_time_limit, num_drones):
        self.routes = []
        self.search_time_limit = search_time_limit #Time in seconds
        self.num_drones =  num_drones
        self.load_assets()

    def load_assets(self):
        self.asset_indexes = np.load("../Data/asset_indexes.npy")
        self.distance_matrix = np.load("../Data/distance_matrix.npy")
        self.photo_indexes = np.load("../Data/photo_indexes.npy")
        self.points_lat_long = np.load("../Data/points_lat_long.npy")
        self.predecessors = np.load("../Data/predecessors.npy")
        self.waypoint_indexes = np.load("../Data/waypoint_indexes.npy")

    #Inits OR-tools and finds the shortest path for X amount of drones starting from inital point
    def find_initial_route(self):
        manager = pywrapcp.RoutingIndexManager(len(self.distance_matrix), self.num_drones, 0)
        routing = pywrapcp.RoutingModel(manager)

        #Defines how OR-tool checks distances between waypoitns
        def distance_callback(from_index: int, to_index: int):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return int(self.distance_matrix[from_node][to_node])

    
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        search_parameters.time_limit.seconds = self.search_time_limit
        search_parameters.log_search = True

        #Start our search from best prev route if already exists
        if self.routes:
            routes_internal = [[manager.NodeToIndex(n) for n in r] for r in self.routes]

            initial_solution = routing.ReadAssignmentFromRoutes(routes_internal, True)

            solution = routing.SolveFromAssignmentWithParameters(initial_solution, search_parameters)

        #Start search from beginning
        else:
            solution = routing.SolveWithParameters(search_parameters)
        self.getRouteList(routing, manager, solution)

    #Resets the waypoints for all drones and populates it with new shortest routes
    def getRouteList(self, routing, manager, solution):
        self.routes = [] 
        for vehicle in range(routing.vehicles()):
            if solution:
                print("Total distance:", solution.ObjectiveValue())
                index = routing.Start(0)
                route = []
                while not routing.IsEnd(index):
                    self.route.append(manager.IndexToNode(index))
                    index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
                self.routes.append(route)
                print("Route:", route)

            else:
                print("No solution found.")

route = RouteFinder()
route.find_initial_route()

Total distance: 248359
Route: [0, 1820, 1867, 1873, 1870, 1819, 1871, 1822, 1872, 1869, 1821, 1868, 1866, 1824, 1865, 1823, 1830, 1829, 1795, 1798, 1797, 1796, 1799, 1802, 1801, 1800, 1807, 1810, 1809, 1808, 2062, 2069, 2070, 2056, 2059, 2048, 2051, 2040, 2043, 2044, 2047, 2052, 2055, 2072, 2075, 2036, 2039, 2032, 2035, 2476, 2142, 2168, 2157, 2156, 2175, 2174, 2153, 2152, 2179, 2178, 2149, 2148, 2132, 2131, 2096, 2095, 2092, 2091, 2086, 2085, 2108, 2107, 2100, 2099, 2104, 2103, 2120, 2119, 2116, 2115, 2112, 2111, 2128, 2127, 2124, 2123, 2122, 2121, 2126, 2125, 2110, 2109, 2114, 2113, 2118, 2117, 2102, 2101, 2098, 2097, 2106, 2105, 2088, 2087, 2084, 2083, 2090, 2089, 2094, 2093, 2130, 2129, 2147, 2146, 2177, 2176, 2151, 2150, 2173, 2172, 2155, 2154, 2171, 2170, 2169, 2167, 2166, 2159, 2160, 2161, 2158, 2141, 2140, 2135, 2134, 2133, 2137, 2138, 2139, 2136, 2163, 2164, 2165, 2162, 2143, 2144, 2145, 2477, 2454, 2456, 2458, 2459, 2461, 2462, 2464, 2465, 2467, 2472, 2474, 2484, 2486, 2481, 